In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Alpha-beta pruning algorithm

### Dependencies

In [ ]:
import chess
import chess.gaviota
import random
import signal
import time
from typing import Union

import import_ipynb
import Util
from Globals import *

#### chess.Board.get_best_move_alphabeta
Finds the best move to make based on the alpha-beta pruning algorithm. This algorithm works as follows:
- Iterate over all legal moves in the current position.
- For each move, find the best possible score after making this move.
    - This is done by calling the alpha-beta pruning function recursively: increasing the current iteration by 1 and switching turns.
    - If a move yields a worse score than the previously examined move in the hierarchy, it cannot be the optimal move. Therefore, such moves are not evaluated further.
- Find the maximum (if it's the AI's turn) or the minimum (if it's other player's/AI's turn) score of all legal moves, alongside the move that was able to reach this optimized state. This is the move that the algorithm recommends.

Optionally, the algorithm uses memoization, which is a type of caching. This works by mapping a board state, plus iteration details, to the corresponding score and move and storing it in a cache dictionary. If, during a later iteration, the same board state is reached on the same iteration, these values can be read from the cache.

__This function is implemented recursively.__

###### <b><u>Arguments</u></b>
__cache__ _(dict)_ : A cache dictionary if memoization is desired, or None if memoization should be disabled.  
__alpha__ _(int)_ : The "alpha" value in the current iteration. On the first iteration, this value should be strongly negative.  
__beta__ _(int)_ : The "beta" value in the current iteration. On the first iteration, this value should be strongly positive.  
__ai_turn__ _(bool)_ : Whether or not the current turn is of the AI that started the search.  
__color__ _(chess.Color)_ : The color of the player whose turn it currently is.  
__endgame_tablebase__ _(Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase])_ : The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  
__iteration__ _(int)_ : The depth of the search (amount of moves currently looking ahead).  
__max_iterations__ _(int)_ : The maximum depth of the search.  
__last_eval_score__ _(int)_ : The score provided by the previous evaluation in the search.  

###### <b><u>Returns <i>(int, chess.Move, bool)</i></u></b>
- The board score after making the recommended best move.
- The recommended best move to make.
- Whether or not the endgame library was used to find the move.
- Whether or not the endgame library was used to find the move.

###### <b><u>Side effects</u></b>
- If a cache dictionary is provided, new values may be added to this dictionary.
- If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def get_best_move_alphabeta(
    self,
    cache: dict,
    alpha: int,
    beta: int,
    ai_turn: bool,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase],
    iteration: int,
    max_iterations: int,
    last_eval_score: int
) -> (int, chess.Move, bool):
                            
    if cache is not None and (iteration, self.get_state_string()) in cache:
        return cache[(iteration, self.get_state_string())]

    original_color = color if ai_turn else not color
    result_score = self.get_search_result_if_finished(original_color, iteration, max_iterations, last_eval_score)
    if result_score is not None:
        return result_score, None, False

    # If the game has not finished, check additional moves using the alpha-beta pruning algorithm
    best_score = Globals.MAX_EVALUATION_SCORE * (-1 if ai_turn else 1)
    best_move = None
    best_move_used_endgame = False
    for move in self.legal_moves:
        eval_score, used_endgame_anywhere = self.evaluate_move(color, not ai_turn, last_eval_score, iteration, move, endgame_tablebase)
        self.push(move)

        if self.is_game_over():
            score_after_move = eval_score
        else:
            score_after_move, _, used_endgame = self.get_best_move_alphabeta(cache, alpha, beta, not ai_turn, not color, endgame_tablebase,
                                                                             iteration + 1, max_iterations, eval_score)
            used_endgame_anywhere = used_endgame_anywhere or used_endgame

        self.pop()
                                                               
        if (ai_turn and score_after_move > best_score) or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
            best_move_used_endgame = used_endgame_anywhere

        if ai_turn:
            if best_score >= beta:
                return best_score, best_move, best_move_used_endgame
            if best_score > alpha:
                alpha = best_score
        else:
            if best_score <= alpha:
                return best_score, best_move, best_move_used_endgame
            if best_score < beta:
                beta = best_score

    if cache is not None:
        cache[(iteration, self.get_state_string())] = best_score, best_move, best_move_used_endgame
        
    return best_score, best_move, best_move_used_endgame

chess.Board.get_best_move_alphabeta = get_best_move_alphabeta
del get_best_move_alphabeta

#### make_move_alphabeta
Finds the best possible move according to the alpha-beta pruning algorithm and pushes it onto the move stack.

###### <b><u>Arguments</u></b>
__board__ _(chess.Board)_ : The board to push the move to.  
__color__ _(chess.Color)_ : The color of the player that makes the move.  
__search_depth__ _(int)_ : The iteration depth of the alpha-beta search.  
__endgame_tablebase__ _(Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase])_ : The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

###### <b><u>Returns <i>(int, chess.Move, bool, bool)</i></u></b>
- The evaluated score of the best possible move.
- The best possible move that was found.
- Whether or not the endgame library was used to find the move.
- Whether or not the endgame library was used to find the move.

###### <b><u>Side effects</u></b>
- The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_alphabeta(
    board: chess.Board,
    color: chess.Color,
    search_depth: int,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]
) -> (int, chess.Move, bool):

    score, move, used_endgame = board.get_best_move_alphabeta(
        cache={}, 
        alpha=-Globals.MAX_EVALUATION_SCORE, 
        beta=Globals.MAX_EVALUATION_SCORE, 
        ai_turn=True, 
        color=color,
        endgame_tablebase=endgame_tablebase,
        iteration=0, 
        max_iterations=search_depth,
        last_eval_score=0 # The starting board score doesn't matter, it's evaluated by score difference
    )
    
    board.push(move)
    return score, move, used_endgame

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>